# 30-121 : Tidy up the Hyperparameter code

## Web References

- [Getting started with KerasTuner](https://keras.io/guides/keras_tuner/getting_started/)
- [Hyper parameter optimisation with Hyperopt and MLflow](https://www.youtube.com/watch?v=W_mJxQupJ4I)
- [hyperopt_logistic_regression.ipynb](https://github.com/amoat7/mlflow_tutorial/blob/master/notebooks/hyperopt_logistic_regression.ipynb)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import logging
from functools import partial

import pandas as pd
import tensorflow as tf
import keras as k
import kerastuner as kt

from competition import source_data as sd
from competition import data_preparation as dp
from competition import feature_engineering as fe
from competition import model_data as md
from competition import model_training as mt
from competition import model_layers as ml
from competition import model_definitions as mm

import mlflow
import mlflow.keras

<ipython-input-2-b0e74d1bd85b>:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
# Set the GPU memory for growth
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Configure Logging

In [4]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2023-03-27 03:12:28 INFO     Started


## Data Collection

In [5]:
# load the source training set
df_source = sd.read_csv('../data/train.csv.gz',
                        compression='gzip',
                        dtype=sd.source_dtype)

(13174211, 20)


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


In [6]:
# load the source training labels
df_source_labels = sd.read_csv('../data/train_labels.csv')

(212022, 2)


,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090314121766812_q1,1


## Data Preparation & Cleaning

In [7]:
# prepare the main dataset
df_source = dp.prepare_main_dataset(df_source,
                                    elapsed_time_min_clip=0,
                                    elapsed_time_max_clip=3691298)

# remove sessions with problems
problem_sessions = dp.find_problem_sessions(df_source)
df_source = df_source[~df_source['session_id'].isin(problem_sessions)]

In [8]:
# prepare the label dataset
df_source_labels = dp.prepare_label_dataset(df_source_labels)

# remove sessions with problems
df_source_labels = df_source_labels[~df_source_labels['session_id'].isin(problem_sessions)]

## Feature Engineering

In [9]:
# create the initial features
df_features = fe.create_initial_features(df_source, df_source_labels)

In [10]:
# add the feature to the features dataset
df_features = fe.add_elapsed_time_features(
    features=df_features,
    X=df_source)

In [11]:
# add the total count features to the features dataset
df_features = fe.add_count_total_features(
    features=df_features,
    X=df_source)

In [12]:
# add the unique count features to the features dataset
df_features = fe.add_count_unique_features(
    features=df_features,
    X=df_source)

In [13]:
# # add the screen heatmap feature to the features dataset
# df_features = fe.add_screen_heatmap_feature(df_features, df_source)

# with pd.option_context('display.max_columns', None):
#     display(df_features.head(6))

## Data Selection

In [14]:
random_state = 51

In [15]:

# split the dataset into train, validation and test sets
train, val, test = md.select_sessions(
    y=df_source_labels,
    random_state=random_state,
    test_size=0.60,
    train_size=0.75)

Train: 3495
Validation: 1165
Test: 6988


## Model Training

### Create Datasets

In [16]:
# set the feature list
feature_list = ['elapsed_time_sum', 'elapsed_time_max', 'elapsed_time_min', 'elapsed_time_mean', 'elapsed_time_mode']

# create the simple model dataset
simple_model_dataset = md.get_feature_dataset(
    features=df_features,
    y=df_source_labels,
    feature_list=feature_list,
    train=train,
    val=val,
    test=test,
    include_question=True,
    expand_question=False)

2023-03-27 03:13:10 INFO     -- Creating the train dataset
2023-03-27 03:13:10 INFO     Creating the dataset for all level groups


  0%|          | 0/62910 [00:00<?, ?it/s]

2023-03-27 03:13:57 INFO     -- Creating the val dataset
2023-03-27 03:13:57 INFO     Creating the dataset for all level groups


  0%|          | 0/20970 [00:00<?, ?it/s]

2023-03-27 03:14:12 INFO     -- Creating the test dataset
2023-03-27 03:14:12 INFO     Creating the dataset for all level groups


  0%|          | 0/125784 [00:00<?, ?it/s]

In [17]:
# define the simple model input shape
input_data = simple_model_dataset['train']['X']
simple_model_shape = input_data.shape[1]
print('simple_model_shape:', simple_model_shape)

# define the output shape
output_data = simple_model_dataset['train']['y']
simple_model_output_shape = 1
print('output_shape', simple_model_output_shape)

simple_model_shape: 23
output_shape 1


### Training

In [39]:
# create the experiment
mlflow.set_experiment("params_03")

<Experiment: artifact_location='file:///workspaces/dsm150-2022-oct/cw02/phase_03/mlruns/260316474676758529', creation_time=1679886945984, experiment_id='260316474676758529', last_update_time=1679886945984, lifecycle_stage='active', name='params_03', tags={}>

#### Simple Dense

In [41]:
mlflow.end_run()

In [42]:
# create the hyperparameter object
def define_tune_parameters(hp):
    hp.Int('dense_layer_count', min_value=1, max_value=20, step=1)
    hp.Int('dense_units', min_value=640, max_value=1700, step=32)
    #hp.Choice('dense_activation', values=['relu', 'tanh', 'LeakyReLU', 'PReLU', 'Swish'])
    hp.Choice('dense_activation', values=['relu', 'tanh', 'LeakyReLU'])
    hp.Float('dense_l1_regulization', min_value=0.0, max_value=0.0005, step=0.00001)
    hp.Float('dense_l2_regulization', min_value=0.0, max_value=0.001, step=0.0001)
    hp.Float('dense_dropout', min_value=0.005, max_value=0.1, step=0.005)
    hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])

# find the best model
model = mm.tune_simple_dense_model(
    define_tune_parameters=define_tune_parameters,
    dataset=simple_model_dataset,
    max_trials=2000,
    input_shape=simple_model_shape,
    output_shape=simple_model_output_shape,
    dense_layer_count='dense_layer_count',
    dense_units='dense_units',
    dense_activation='dense_activation',
    dense_l1_regulization='dense_l1_regulization',
    dense_l2_regulization='dense_l2_regulization',
    dense_dropout='dense_dropout',
    train_epochs=50,
    train_batch_size=4000,
    train_optimizer=k.optimizers.Adam,
    train_learning_rate='learning_rate',
    train_loss='binary_crossentropy',
    train_metrics=['accuracy'],
    train_class_weight=None)

model.summary()


Trial 232 Complete [00h 00m 45s]
val_accuracy: 0.7425369620323181

Best val_accuracy So Far: 0.7476394772529602
Total elapsed time: 02h 33m 27s
2023-03-27 06:09:55 INFO     on_trial_begin



Search: Running Trial #233

Value             |Best Value So Far |Hyperparameter
16                |4                 |dense_layer_count
1696              |1056              |dense_units
tanh              |relu              |dense_activation
0.00049           |2e-05             |dense_l1_regulization
0.0002            |0.0005            |dense_l2_regulization
0.06              |0.055             |dense_dropout
1e-05             |0.001             |learning_rate

2023-03-27 06:09:55 INFO     Creating simple dense model
Epoch 1/50
 6/16 [==========>...................] - ETA: 0s - loss: 450.3367 - accuracy: 0.5590WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0270s vs `on_train_batch_end` time: 0.0462s). Check your callbacks.
2023-03-27 06:09:57 WARNING  Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0270s vs `on_train_batch_end` time: 0.0462s). Check your callbacks.
16/16 [===============